In [1]:
# Import Libraries
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

In [2]:
# Use the Chrome Driver
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\bentl\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## NASA Mars News

* Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that can be referenced later.

In [3]:
# URL of page to be scraped
url_news = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url_news)
# Create BeautifulSoup object; parse with 'lxml'
soup_news = BeautifulSoup(response.text, 'lxml')

In [ ]:
# Check the content to see what can be scrapped
print(soup_news.prettify())

In [4]:
news_title = soup_news.find('div', class_='content_title').text
news_p = soup_news.find("div", class_="rollover_description_inner").text

print(news_title)
print(news_p)




NASA Ingenuity Mars Helicopter Prepares for First Flight



Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  



## JPL Mars Space Images - Featured Image
* Search performed on Featured Iamge tag

In [5]:
url_image = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'

browser.visit(url_image)
time.sleep(1)

# Use B.Soup to scrape the page
html= browser.html
soup = BeautifulSoup(html, "html.parser")

browser.links.find_by_partial_text('FULL IMAGE')


In [6]:
for link in soup.find_all('img'):
    if(link.get('src').startswith( 'image/featured')):
      featured_image_url = f"https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{link.get('src')}"

In [7]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


## Mars Facts
* Read in Web Site and find the facts table
* Convert to Pandas table 
* Convert Pandas table to a HTML table

In [8]:
url_facts = 'https://space-facts.com/mars/'

tables = pd.read_html(url_facts)
facts_table = pd.DataFrame(tables[0])

facts_table.rename(columns = {0: "Fact", 1: "Value"}, inplace=True)

facts_table.set_index("Fact", inplace=True)
facts_table

,Value
Fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [ ]:
# Create html Table
table_html = facts_table.to_html()

print(table_html)

## Mars Hemispheres
* Obtain high resolution images for each of Mar's hemispheres
* Goto the index page and scrape all the product pages for each hemisphere 
* Then scrape the image from each product page

In [9]:
url_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url_hemispheres)

In [10]:
html = browser.html
soup_hemispheres = BeautifulSoup(html, 'html.parser')

products = soup_hemispheres.find_all('div', class_='item')

products_links = []

for product in products:
    heading = product.find('h3').text
    product_link = \
    f"https://astrogeology.usgs.gov{product.find('a', class_='itemLink product-item')['href']}"
    
    products_links.append({"title" : heading, "img_url" : product_link})

print(products_links)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]


In [11]:
# Now that we have the page for each product, scrape the image link for each product 

image_links = []

for hemisphere in products_links:
    
    browser.visit(hemisphere['img_url'])
    html = browser.html
    soup_hemisphere = BeautifulSoup(html, 'html.parser')

    photo_url = \
        f"https://astrogeology.usgs.gov{soup_hemisphere.find('img', class_='wide-image')['src']}"

    image_links.append({"title" : hemisphere['title'], "img_url" : photo_url})

print(image_links)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
